In [87]:
import numpy as np
def join(lst1, lst2):
    return [(i, j) for i in lst1 for j in lst2 if i == j]

In [81]:
L1 = [2] + [1] * 500
L2 = [1] + [2] * 500
L1_str = ", ".join([str(x) for x in L1[:5]]) + '...' + ", ".join([str(x) for x in L1[-3:]])
L2_str = ", ".join([str(x) for x in L2[:5]]) + '...' + ", ".join([str(x) for x in L2[-3:]])
print(f"{L1_str} ({len(L1)})")
print(f"{L2_str} ({len(L2)})")
#print(f"{L2[:10]}...{L2[:-5]}")

2, 1, 1, 1, 1...1, 1, 1 (501)
1, 2, 2, 2, 2...2, 2, 2 (501)


In [49]:
join_results = join(L1,L2)

In [79]:
import random
sampled = random.sample(join_results ,100)

In [80]:
from collections import Counter
Counter([x[0] for x in sampled])

Counter({1: 51, 2: 49})

In [52]:
len(join_results)

1000

stratergy a: sample one value from R1, join with only one in R2, and mark that R1 can't be sampled again

In [142]:
def sample_join_method_a(table1, table2, sample_size=50):
    table1 = np.array(table1) if not isinstance(table1, np.ndarray) else table1
    table2 = np.array(table2) if not isinstance(table2, np.ndarray) else table2
    random_join = []
    max_int = len(table1) - 1
    already_visited_table1 = set()

    while len(random_join) < sample_size and len(already_visited_table1) < len(table1):
        next_rand_index = random.randint(0, max_int)
        while next_rand_index in already_visited_table1:
            next_rand_index = random.randint(0, max_int)

        already_visited_table1.add(next_rand_index)
        value_from_rand_index = table1[next_rand_index]

        joined_indexes_from_table2 = np.where(table2 == value_from_rand_index)
        if not joined_indexes_from_table2:
            continue
        rand_matched_index_table2 = random.choice(joined_indexes_from_table2)[0]
        #if rand_matched_index_table2.size:
        rand_value_table2 = table2[rand_matched_index_table2]
        random_join.append((value_from_rand_index, rand_value_table2))

    print (Counter([x[0] for x in random_join]))

In [144]:
sample_join_method_a(L1,L2,1000)
# the problem - until we sampled "2", we brought only with 1 matching.

Counter({1: 500, 2: 1})


stratergy b: like a, but using Olken rejectness test: duplication allowed!

In [153]:
def sample_join_method_b(table1, table2, sample_size=50):
    table1 = np.array(table1) if not isinstance(table1, np.ndarray) else table1
    table2 = np.array(table2) if not isinstance(table2, np.ndarray) else table2
    random_join = []
    max_int = len(table1) - 1
    already_visited_table1 = set()

    table_2_counter = Counter(table2)
    max_cardinality = table_2_counter.most_common(1)[0][1]

    while len(random_join) < sample_size and len(already_visited_table1) < len(table1):
        next_rand_index = random.randint(0, max_int)
        while next_rand_index in already_visited_table1:
            next_rand_index = random.randint(0, max_int)

        value_from_rand_index = table1[next_rand_index]

        joined_indexes_from_table2 = np.where(table2 == value_from_rand_index)
        if not joined_indexes_from_table2:
            continue

        olken_accept_test = table_2_counter[value_from_rand_index] / max_cardinality

        if np.random.random_sample() >= olken_accept_test:
            continue

        # already_visited_table1.add(next_rand_index)

        rand_matched_index_table2 = random.choice(joined_indexes_from_table2)[0]

        rand_value_table2 = table2[rand_matched_index_table2]
        random_join.append((value_from_rand_index, rand_value_table2))

    print (Counter([x[0] for x in random_join]))


In [154]:
sample_join_method_b(L1, L2, 1000)

Counter({2: 505, 1: 495})


In [155]:
join_results = join(L1,L2)

In [156]:
len(join_results)

1000

stratergy c: We sampling alternately from R1 and R2, and each table maintain how many times each value can be selected, using statistics.